In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
df_city=pd.read_pickle("cleaned_city_bookings.pkl")
df_city.head(15)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,arrival_date,country_grouped,lead_time_scaled,required_car_parking_spaces_scaled,special_requests_clipped,special_requests_scaled,deposit_paid,got_requested_room_type
40060,City Hotel,0,6,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,0.00,0,0,Check-Out,2015-07-03,2015-07-01,PRT,-0.931250,-0.256841,0,-0.728703,0,1
40061,City Hotel,1,88,2015,July,27,1,0,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.50,0,1,Canceled,2015-07-01,2015-07-01,PRT,-0.147315,-0.256841,1,0.554576,0,1
40062,City Hotel,1,65,2015,July,27,1,0,4,1,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,68.00,0,1,Canceled,2015-04-30,2015-07-01,PRT,-0.367199,-0.256841,1,0.554576,0,1
40063,City Hotel,1,92,2015,July,27,1,2,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.50,0,2,Canceled,2015-06-23,2015-07-01,PRT,-0.109074,-0.256841,2,1.837855,0,1
40064,City Hotel,1,100,2015,July,27,2,0,2,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.50,0,1,Canceled,2015-04-02,2015-07-02,PRT,-0.032592,-0.256841,1,0.554576,0,1
40065,City Hotel,1,79,2015,July,27,2,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.50,0,1,Canceled,2015-06-25,2015-07-02,PRT,-0.233356,-0.256841,1,0.554576,0,1
40066,City Hotel,0,3,2015,July,27,2,0,3,1,0.0,0,HB,PRT,Groups,TA/TO,0,0,0,A,A,1,No Deposit,1.0,NaN,0,Transient-Party,58.67,0,0,Check-Out,2015-07-05,2015-07-02,PRT,-0.959930,-0.256841,0,-0.728703,0,1
40067,City Hotel,1,63,2015,July,27,2,1,3,1,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,68.00,0,0,Canceled,2015-06-25,2015-07-02,PRT,-0.386319,-0.256841,0,-0.728703,0,1
40068,City Hotel,1,62,2015,July,27,2,2,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,8.0,NaN,0,Transient,76.50,0,1,No-Show,2015-07-02,2015-07-02,PRT,-0.395879,-0.256841,1,0.554576,0,1
40069,City Hotel,1,62,2015,July,27,2,2,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,8.0,NaN,0,Transient,76.50,0,1,No-Show,2015-07-02,2015-07-02,PRT,-0.395879,-0.256841,1,0.554576,0,1


In [4]:
features = [
    'lead_time_scaled',
    'required_car_parking_spaces_scaled',
    'special_requests_scaled',
    'market_segment',
    'distribution_channel',
    'deposit_paid',
    'got_requested_room_type',
    'customer_type',
    'country_grouped'
    ]
target = 'is_canceled'
X = df_city[features]
y = df_city[target]
X_encoded = pd.get_dummies(X, columns=['market_segment','distribution_channel','deposit_paid','got_requested_room_type','customer_type','country_grouped'], drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [5]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)[:,1]
print("AUC:", roc_auc_score(y_test, y_pred_proba))

AUC: 0.8777468345415733


In [6]:
df_test = X_test.copy()
df_test['storno_score'] = y_pred_proba

In [7]:
df_test.head(20)

,lead_time_scaled,required_car_parking_spaces_scaled,special_requests_scaled,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_TA/TO,deposit_paid_1,got_requested_room_type_1,customer_type_Transient,customer_type_Transient-Party,country_grouped_BRA,country_grouped_DEU,country_grouped_ESP,country_grouped_FRA,country_grouped_GBR,country_grouped_IRL,country_grouped_ITA,country_grouped_NLD,country_grouped_Other,country_grouped_PRT,storno_score
43276,-0.759166,-0.256841,0.554576,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,0.162009
108193,-0.806967,-0.256841,-0.728703,False,False,False,False,False,True,False,True,False,True,True,False,False,False,False,False,False,False,False,False,True,False,0.480496
114618,0.416736,-0.256841,-0.728703,False,False,False,False,False,True,False,True,False,True,True,False,False,False,False,False,True,False,False,False,False,False,0.595419
100416,0.187292,-0.256841,-0.728703,False,False,False,False,False,True,False,True,False,True,True,False,False,False,False,True,False,False,False,False,False,False,0.503123
46406,0.426296,-0.256841,-0.728703,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,0.998032
48254,2.768542,-0.256841,-0.728703,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,0.999478
60234,1.659560,-0.256841,-0.728703,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,0.999022
115644,-0.137754,-0.256841,0.554576,False,False,True,False,False,False,True,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,0.088827
98106,0.359375,-0.256841,0.554576,False,False,False,False,False,True,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,0.078266
45535,-0.223796,-0.256841,-0.728703,False,False,False,True,False,False,False,True,True,True,True,False,False,False,False,False,False,False,False,False,False,True,0.997156
